In [1]:
import tensorflow as tf
from tensorflow import keras

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
housing = fetch_california_housing()

In [3]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [4]:
# モデル構築
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

# コンパイル
model.compile(loss="mean_squared_error", optimizer="sgd")

history = model.fit(
    X_train, 
    y_train,
    epochs=20,
    validation_data=(X_valid, y_valid)
)

mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 1.3545 - val_loss: 0.5609
Epoch 2/20
363/363 [==============================] - 0s 442us/step - loss: 0.5797 - val_loss: 0.4827
Epoch 3/20
363/363 [==============================] - 0s 440us/step - loss: 0.4997 - val_loss: 0.4513
Epoch 4/20
363/363 [==============================] - 0s 442us/step - loss: 0.4627 - val_loss: 0.4407
Epoch 5/20
363/363 [==============================] - 0s 438us/step - loss: 0.4695 - val_loss: 0.4446
Epoch 6/20
363/363 [==============================] - 0s 420us/step - loss: 0.4405 - val_loss: 0.4148
Epoch 7/20
363/363 [==============================] - 0s 421us/step - loss: 0.4453 - val_loss: 0.4152
Epoch 8/20
363/363 [==============================] - 0s 417us/step - loss: 0.4228 - val_loss: 0.4068
Epoch 9/20
363/363 [==============================] - 0s 415us/step - loss: 0.4419 - val_loss: 0.4073
Epoch 10/20
363/363 [==============================] - 0s 424us/step - loss: 0.4275 

# 関数型APIを使ったモデルの構築
### 非シーケンシャルニューラルネットワーク

- 入力の全てまたは一部を直接出力層に接続する。

In [5]:
input_ = keras.layers.Input(shape=X_train.shape[1:])   # 入力タイプを指定
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

In [6]:
# 2つのパスを連結
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

In [7]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit(
    (X_train_A, X_train_B),
    y_train, 
    epochs=20,
    validation_data=((X_valid_A, X_valid_B), y_valid)
)

mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

Epoch 1/20
363/363 [==============================] - 0s 624us/step - loss: 2.5746 - val_loss: 0.9179
Epoch 2/20
363/363 [==============================] - 0s 462us/step - loss: 0.8468 - val_loss: 0.6782
Epoch 3/20
363/363 [==============================] - 0s 459us/step - loss: 0.7074 - val_loss: 0.6375
Epoch 4/20
363/363 [==============================] - 0s 461us/step - loss: 0.6514 - val_loss: 0.6039
Epoch 5/20
363/363 [==============================] - 0s 460us/step - loss: 0.6428 - val_loss: 0.5808
Epoch 6/20
363/363 [==============================] - 0s 465us/step - loss: 0.6127 - val_loss: 0.5600
Epoch 7/20
363/363 [==============================] - 0s 459us/step - loss: 0.5996 - val_loss: 0.5419
Epoch 8/20
363/363 [==============================] - 0s 458us/step - loss: 0.5775 - val_loss: 0.5271
Epoch 9/20
363/363 [==============================] - 0s 458us/step - loss: 0.5619 - val_loss: 0.5197
Epoch 10/20
363/363 [==============================] - 0s 457us/step - loss: 0.531

In [8]:
# 補助出力の追加
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

In [9]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")   # メイン出力の損失に大きな重みを与える。

history = model.fit(
    [X_train_A, X_train_B],
    [y_train, y_train], 
    epochs=20,
    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid])
)

total_loss, main_loss, aux_loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model.predict((X_new_A, X_new_B))

Epoch 1/20
363/363 [==============================] - 1s 811us/step - loss: 1.8773 - main_output_loss: 1.7421 - aux_output_loss: 3.0948 - val_loss: 0.5516 - val_main_output_loss: 0.4808 - val_aux_output_loss: 1.1890
Epoch 2/20
363/363 [==============================] - 0s 564us/step - loss: 0.5629 - main_output_loss: 0.5014 - aux_output_loss: 1.1165 - val_loss: 0.5032 - val_main_output_loss: 0.4489 - val_aux_output_loss: 0.9920
Epoch 3/20
363/363 [==============================] - 0s 563us/step - loss: 0.5267 - main_output_loss: 0.4801 - aux_output_loss: 0.9466 - val_loss: 0.4927 - val_main_output_loss: 0.4494 - val_aux_output_loss: 0.8824
Epoch 4/20
363/363 [==============================] - 0s 564us/step - loss: 0.4992 - main_output_loss: 0.4608 - aux_output_loss: 0.8451 - val_loss: 0.4580 - val_main_output_loss: 0.4228 - val_aux_output_loss: 0.7751
Epoch 5/20
363/363 [==============================] - 0s 568us/step - loss: 0.4730 - main_output_loss: 0.4428 - aux_output_loss: 0.7454 

## サブクラス化APIを使用したダイナミックなモデルの構築
- ループを含んだり、形を変更したり、条件分岐したりといったダイナミックな動作を必要とするクラスを作成できる。
- モデルのアーキテクチャがメソッド内に隠れてしまうため、Kerasは簡単にモデルを精査できないし、保存、クローン作成もできない。
- summary()メソッドを呼び出してもメソッドを呼び出しても、層のリストが得られるだけで、層がどのように接続されているかについての情報は得られない。

In [10]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatnate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output
    
# model = WideAndDeepModel()

In [11]:
# モデルの保存
model.save("models/my_keras_model.h5")

In [12]:
# モデルをロード
model = keras.models.load_model("models/my_keras_model.h5")

# コールバック
- 訓練の開始、終了時、各エポックの開始、終了時、さらにバッチを１つ処理する前後にKerasが呼び出すオブジェクトのリストを指定できるようになっている。
- モデルの訓練中、定期的な間隔で（デフォルトでは各エポック終了後）モデルのチェックポイントを保存する。

In [13]:
# モデル構築
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

# コンパイル
model.compile(loss="mean_squared_error", optimizer="sgd")

checkpoint_cb = keras.callbacks.ModelCheckpoint("models/my_keras_model_callback.h5", save_best_only=True)   # 検証セットに対する性能が最高になった時だけモデルを保存する。

history = model.fit(
    X_train, 
    y_train,
    epochs=20,
    validation_data=(X_valid, y_valid),
    callbacks=[checkpoint_cb]
)

Epoch 1/20
363/363 [==============================] - 0s 558us/step - loss: 1.7458 - val_loss: 0.5306
Epoch 2/20
363/363 [==============================] - 0s 405us/step - loss: 1.3604 - val_loss: 0.4311
Epoch 3/20
363/363 [==============================] - 0s 405us/step - loss: 0.4910 - val_loss: 0.4171
Epoch 4/20
363/363 [==============================] - 0s 406us/step - loss: 0.4357 - val_loss: 0.3950
Epoch 5/20
363/363 [==============================] - 0s 409us/step - loss: 0.4125 - val_loss: 0.3864
Epoch 6/20
363/363 [==============================] - 0s 405us/step - loss: 0.4038 - val_loss: 0.4381
Epoch 7/20
363/363 [==============================] - 0s 412us/step - loss: 0.4261 - val_loss: 0.3805
Epoch 8/20
363/363 [==============================] - 0s 434us/step - loss: 0.3977 - val_loss: 0.3743
Epoch 9/20
363/363 [==============================] - 0s 402us/step - loss: 0.3856 - val_loss: 0.3675
Epoch 10/20
363/363 [==============================] - 0s 410us/step - loss: 0.397

In [14]:
# 早期打ち切り
# patience引数で指定されたエポック数だけ検証セットに対する性能が上がらない時に訓練を中止し、オプションで最良のモデルにロールバックする。
# 性能が上がらなくなった時に、訓練を早期打ち切りする。（時間とリソースの浪費を防ぐため）

# モデル構築
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

# コンパイル
model.compile(loss="mean_squared_error", optimizer="sgd")

checkpoint_cb = keras.callbacks.ModelCheckpoint("models/my_keras_model_callback.h5", save_best_only=True)   # 検証セットに対する性能が最高になった時だけモデルを保存する。

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(
    X_train, 
    y_train,
    epochs=20,
    validation_data=(X_valid, y_valid),
    callbacks=[checkpoint_cb, early_stopping_cb]
)

Epoch 1/20
363/363 [==============================] - 0s 560us/step - loss: 1.0704 - val_loss: 0.5615
Epoch 2/20
363/363 [==============================] - 0s 411us/step - loss: 0.6089 - val_loss: 0.4724
Epoch 3/20
363/363 [==============================] - 0s 409us/step - loss: 0.5338 - val_loss: 0.4511
Epoch 4/20
363/363 [==============================] - 0s 407us/step - loss: 0.4718 - val_loss: 0.4405
Epoch 5/20
363/363 [==============================] - 0s 411us/step - loss: 0.4584 - val_loss: 0.4253
Epoch 6/20
363/363 [==============================] - 0s 410us/step - loss: 0.4500 - val_loss: 0.4218
Epoch 7/20
363/363 [==============================] - 0s 410us/step - loss: 0.4315 - val_loss: 0.4143
Epoch 8/20
363/363 [==============================] - 0s 409us/step - loss: 0.4439 - val_loss: 0.4031
Epoch 9/20
363/363 [==============================] - 0s 405us/step - loss: 0.4368 - val_loss: 0.4057
Epoch 10/20
363/363 [==============================] - 0s 412us/step - loss: 0.441

In [15]:
# 過学習を検知するカスタムコールバック関数
class PrintValTrainRationCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

# TensorBoardで可視化
- 訓練中に学習曲線を表示したり、複数の実行で学習曲線を比較したり、計算グラフを可視化したり、訓練の統計情報を解析したり、モデルが生成した画像を表示したり、複雑な多次元データを３じげんに射影したり、自動的にクラスタリングしたり、その他様々なことのために使用できる優れた対話的可視化ツール。
- 可視化したいデータをイベントファイルと呼ばれる特別なバイナリファイルに出力するようにプログラムを書き換える必要がある。
- TensorBoardサーバーはログディレクトリを監視し、自動的に変換箇所を取り出して可視化データを更新。
    - 訓練中の学習曲線を表示できる。

In [16]:
import os 
root_loggir = os.path.join(os.curdir, "models/my_logs")

def get_run_loggir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_loggir, run_id)

run_loggir = get_run_loggir()

# Keras
# モデル構築
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

# コンパイル
model.compile(loss="mean_squared_error", optimizer="sgd")

checkpoint_cb = keras.callbacks.ModelCheckpoint("models/my_keras_model_callback.h5", save_best_only=True)   # 検証セットに対する性能が最高になった時だけモデルを保存する。

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

tensorboard_cb = keras.callbacks.TensorBoard(run_loggir)

history = model.fit(
    X_train, 
    y_train,
    epochs=20,
    validation_data=(X_valid, y_valid),
    callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb]
)

Epoch 1/20
363/363 [==============================] - 0s 578us/step - loss: 1.1690 - val_loss: 0.5517
Epoch 2/20
363/363 [==============================] - 0s 412us/step - loss: 0.5632 - val_loss: 0.5280
Epoch 3/20
363/363 [==============================] - 0s 408us/step - loss: 0.4925 - val_loss: 0.4401
Epoch 4/20
363/363 [==============================] - 0s 411us/step - loss: 0.4576 - val_loss: 0.4332
Epoch 5/20
363/363 [==============================] - 0s 408us/step - loss: 0.4707 - val_loss: 0.4218
Epoch 6/20
363/363 [==============================] - 0s 416us/step - loss: 0.4670 - val_loss: 0.4169
Epoch 7/20
363/363 [==============================] - 0s 413us/step - loss: 0.4831 - val_loss: 0.4117
Epoch 8/20
363/363 [==============================] - 0s 409us/step - loss: 0.4607 - val_loss: 0.3994
Epoch 9/20
363/363 [==============================] - 0s 409us/step - loss: 0.4542 - val_loss: 0.3999
Epoch 10/20
363/363 [==============================] - 0s 416us/step - loss: 0.441

In [17]:
# !tensorboard --logdir=./my_logs --port=6006
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

In [18]:
# スカラ、ヒストグラム、イメージ、オーディオ、テキストをロギングするためのコンテキスト

import numpy as np
test_loggir = get_run_loggir()
writer = tf.summary.create_file_writer(test_loggir)
with writer.as_default():
    for step in range(1, 1000 + 1):
        tf.summary.scalar("my_scalar", np.sin(step / 10), step=step)
        data = (np.random.randn(100) + 2) * step / 100   # 何らかの無作為なデータ
        tf.summary.histogram("my_hist", data, buckets=50, step=step)
        images = np.random.rand(2, 32, 32, 3)   # 無作為な32x32 RGB画像
        tf.summary.image("my_images", images * step / 1000, step=step)
        texts = ["The step is " + str(step), "Its square is " + str(step**2)]
        tf.summary.text("my_text", texts, step=step)
        sine_wave = tf.math.sin(tf.range(12000) / 48000 * 2 * np.pi * step)
        audio = tf.reshape(tf.cast(sine_wave, tf.float32), [1, -1, 1])
        tf.summary.audio("my_audio", audio, sample_rate=48000, step=step)

In [19]:
# !tensorboard --logdir=./my_logs --port=6006
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 7842), started 0:00:06 ago. (Use '!kill 7842' to kill it.)